In [1]:
import numpy as np
A = np.array([[0.9512, 0],
              [0.0476, 0.9512]], ndmin = 2)
B = np.array([[1], [2]], ndmin = 2)
C = np.array([0, 1], ndmin = 2)

mean_w = np.array([[0], [0]], ndmin = 2)
Q = 1e-3 * np.array([[9.506, 0.234],
                              [0.234, 9.512]], ndmin = 2)
mean_v = 0
R = 0.0125

h = 0.1 # sampling period in second

mu_0 = np.array([[5], [5]], ndmin = 2)
P_0 = np.array([[1, 0],
                        [0, 1]], ndmin = 2)

print(A)
print(B)
print(C)
print(mean_w)
print(variance_w)

u = np.loadtxt("./Data_linear_case/input_linear_case.txt", skiprows = 1, usecols = 0)
y = np.loadtxt("./Data_linear_case/Measured_output_linear_case.txt", skiprows = 1, usecols = 0)
true_x1 = np.loadtxt("./Data_linear_case/True_state_x1_linear_case.txt", skiprows = 1, usecols = 0)
true_x2 = np.loadtxt("./Data_linear_case/True_state_x2_linear_case.txt", skiprows = 1, usecols = 0)

assert len(u) == len(y) == len(true_x1) == len(true_x2)

number_step_max = len(u)

[[0.9512 0.    ]
 [0.0476 0.9512]]
[[1]
 [2]]
[[0 1]]
[[0]
 [0]]
[[0.009506 0.000234]
 [0.000234 0.009512]]


# Forecast step

In [ ]:
for i in range(its_something):
    mu_tilde[i] = A@mu_tilde[i-1] + B@u[i]
    p_tilde_k = A@P[i]@A.T + Q

# Update step